In [1]:
import pandas as pd
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import math
import numpy as np
import sys
import seaborn as sns
import matplotlib.pyplot as plt
sys.path.insert(0, '..')

from src.constants import (
    COLUMNS_TO_CLEAN,
    CATEGORICAL_COLUMNS,
    DESCRIPTIONS
)
from src.data_analysis import create_treatment_combination
from src.data_cleaning import clean_columns


### Load data

In [ ]:
df_bill_amount = pd.read_csv('data/bill_amount.csv')
df_bill_id = pd.read_csv('data/bill_id.csv')
df_clinical_data = pd.read_csv('data/clinical_data.csv')
df_demographics = pd.read_csv('data/demographics.csv')

### Merging datasets
#### I am merging the df_bill_amount and df_bill_id into one df called df_patient_billing.

In [ ]:
df_patient_billing = (
    df_bill_id.merge(df_bill_amount, on="bill_id", how='left') 
) 
df_patient_billing['date_of_admission'] = pd.to_datetime(df_patient_billing['date_of_admission'])

#### Merging clinical data and demographics

In [ ]:
df_patient_dataset = (df_clinical_data.merge(df_demographics, left_on='id', right_on='patient_id', how='left'))
df_patient_dataset = df_patient_dataset.drop(columns=['patient_id'])
df_patient_dataset = df_patient_dataset.rename(columns={'id': 'patient_id'})

df_patient_dataset['date_of_admission'] = pd.to_datetime(df_patient_dataset['date_of_admission'])
df_patient_dataset['date_of_discharge'] = pd.to_datetime(df_patient_dataset['date_of_discharge'])

In [ ]:
## Printing unique values in race, gender and resident_status 
print(df_patient_dataset['race'].unique())
print(df_patient_dataset['gender'].unique())
print(df_patient_dataset['resident_status'].unique())

### Data cleaning 


In [ ]:
# cleaning numerical columns
df_patient_dataset = clean_columns(df_patient_dataset, COLUMNS_TO_CLEAN)

# cleaning gender and race and resident status
df_patient_dataset['gender'] = df_patient_dataset['gender'].replace({'m': 'Male', 'f': 'Female'})
df_patient_dataset['race'] = df_patient_dataset['race'].replace({'chinese': 'Chinese', 'India': 'Indian'})
df_patient_dataset['resident_status'] = df_patient_dataset['resident_status'].replace({'Singapore citizen': 'Singaporean'})

# creating a new column called age 
df_patient_dataset['date_of_birth'] = pd.to_datetime(df_patient_dataset['date_of_birth'])  
df_patient_dataset['patient_age'] = (df_patient_dataset['date_of_admission'] - df_patient_dataset['date_of_birth']).dt.days // 365

In [ ]:
# Filter df_patient_billing for relevant records
relevant_bills = df_patient_billing[
    df_patient_billing.apply(lambda row: any(
        (df_patient_dataset['patient_id'] == row['patient_id']) & 
        (df_patient_dataset['date_of_admission'] <= row['date_of_admission']) & 
        (df_patient_dataset['date_of_discharge'] >= row['date_of_admission'])
    ), axis=1)
]

# Group by patient_id and date_of_admission, and sum amounts
grouped_bills = relevant_bills.groupby(['patient_id', 'date_of_admission'])['amount'].sum().round(3).reset_index()

# Merge the grouped data back into df_patient_dataset
df_patient_dataset = pd.merge(df_patient_dataset, grouped_bills, on=['patient_id', 'date_of_admission'], how='left').fillna(0)
df_patient_dataset = df_patient_dataset.rename(columns={'amount': 'billing_amount'})

# Creating age groups
bins = [0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100]
labels = ['0-10', '11-20', '21-30', '31-40', '41-50', '51-60', '61-70', '71-80', '81-90', '91-100']
df_patient_dataset['age_group'] = pd.cut(df_patient_dataset['patient_age'], bins=bins, labels=labels, right=False)

In [ ]:
# Count the number of times billing_amount is 0
count_zero_billing = (df_patient_dataset['billing_amount'] == 0).sum()
print(f'The number of times billing_amount is 0: {count_zero_billing}')

count = (df_patient_dataset['date_of_discharge'] < df_patient_dataset['date_of_admission']).sum()
print(f'The number of times date of discharge is before admission: {count}')

# Count the number of unique patient IDs in the dataset
unique_patient_ids = df_patient_dataset['patient_id'].nunique()
print("Number of unique patient IDs:", unique_patient_ids)

# Count occurrences of each patient_id
patient_id_counts = df_patient_dataset['patient_id'].value_counts()

# Count how many patient_ids appear multiple times and how many appear only once
multiple_records = (patient_id_counts > 1).sum()
single_record = (patient_id_counts == 1).sum()

print(f"Patient IDs with multiple records: {multiple_records}")
print(f"Patient IDs with a single record: {single_record}")

In [ ]:
### Stacking to reduce the number of lines for multiple lines for a patient 
# List of columns for which 'first' should be used
columns_use_last = ['gender', 'race', 'resident_status', 'date_of_birth', 'age_group']

# Create the aggregation dictionary
agg_dict = {col: 'last' for col in columns_use_last}
agg_dict.update({col: 'max' for col in df_patient_dataset.columns if col not in columns_use_last})

# Group by 'patient_id' and aggregate
df_patient_dataset = df_patient_dataset.groupby('patient_id').agg(agg_dict)

# Rename the index (which is currently 'patient_id') before resetting it
df_patient_dataset.index.rename('patient_index', inplace=True)

# Now reset the index; 'patient_id' will be added as a column without conflict
df_patient_dataset.reset_index(inplace=True)

## Drop the index
df_patient_dataset.reset_index(drop=True, inplace=True)
df_patient_dataset.rename(columns={'patient_index': 'patient_id'}, inplace=True)

**Data Cleaning**

**Overview:**

Our data analysis focused on a dataset containing patient records, with an emphasis on billing information, admission and discharge dates, and patient demographics. The dataset originally consisted of 3000 unique patient IDs with varying levels of detail.

1. **Billing Data Availability**: A significant portion of the records (1877 instances) had a billing amount of zero. This suggests that billing data is not consistently available across the dataset.

2. **Inconsistencies in Admission and Discharge Dates**: There were 1206 instances where the discharge date preceded the admission date, indicating discrepancies in these records.

3. **Patient Record Distribution**: Out of the 3000 unique patient IDs, a majority (2621) had only a single record associated with them. The remaining 379 patients had multiple records.

**Data Processing and Analysis Steps:**

1. **Parsing Dates**: We encountered issues with parsing `date_of_admission` and `date_of_discharge` fields due to inconsistent formats. To resolve this, each element was parsed individually using `dateutil`, although specifying a format could improve consistency.

2. **Data Aggregation Strategy - 'Trumping' Method**:
   - To streamline the dataset and address the issue of multiple records per patient, we implemented a 'trumping' method. This approach ensured that each patient was represented by a single consolidated record.
   - For numerical columns, we chose to use the maximum value where multiple records existed for a patient. This approach aimed to retain the most significant numerical data for each patient.
   - For non-numerical columns such as 'gender', 'race', 'resident_status', 'date_of_birth', and 'age_group', we decided to use the last value in the series of records. This choice was based on the assumption that the most recent non-numerical information would be the most accurate.
   - We aggregated the data using the specified rules, grouping by 'patient_id'.


## Plots to view data

In [ ]:
subplot_titles = [f'{col}: {DESCRIPTIONS.get(col, "")}' for col in CATEGORICAL_COLUMNS]

num_columns = 2
num_rows = math.ceil(len(CATEGORICAL_COLUMNS) / num_columns)
fig = make_subplots(rows=num_rows, cols=num_columns, subplot_titles=subplot_titles)
for i, col in enumerate(CATEGORICAL_COLUMNS, start=1):
    row = math.ceil(i / num_columns)
    col_in_row = (i-1) % num_columns + 1

    data = df_patient_dataset[col].value_counts()
    fig.add_trace(
        go.Bar(x=data.index, y=data.values, name=col),
        row=row, col=col_in_row
    )

fig.update_layout(
    height=300*num_rows, 
    width=1000, 
    title_text="Distribution of Categorical Columns",
    showlegend=False)
fig.show()


In [ ]:
## HeatMap

numerical_columns = ['cgis_adm', 'cgis_dis', 'gaf_lv', 'weight', 'height'] 
correlation_matrix = df_patient_dataset[numerical_columns].corr()

# Plotting the heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm')
plt.title('Correlation Matrix of Numerical Variables')
plt.show()

### Research Question
#### Effectiveness of Combined Treatments

**Question:** "Does co-prescribing adjunctive medications (e.g., antipsychotics, anticonvulsants, lithium) alongside antidepressants lead to better outcomes in MDD patients compared to antidepressants alone?"
#### Analysis Approach:
**Grouping:** Divide the dataset into two groups - patients receiving only antidepressants (trt_adt) and patients receiving antidepressants along with adjunctive medications (trt_anx, trt_con, trt_oth).

**Outcome Measures:** Use the 'cgis_adm' and 'cgis_dis' scores to assess severity at admission and discharge. Improvement can be measured as the difference between these scores.

In [ ]:

df_patient_dataset['cgis_change'] = df_patient_dataset['cgis_dis'] - df_patient_dataset['cgis_adm']

# Define the two groups
# Group 1: Patients on antidepressants alone
df_patient_dataset['group'] = 'Antidepressants Only'
df_patient_dataset.loc[(df_patient_dataset['trt_adt'] == 1) 
                       & ((df_patient_dataset['trt_anx'] == 1) 
                          | (df_patient_dataset['trt_con'] == 1) |
                            (df_patient_dataset['trt_ssr'] == 1) | 
                            (df_patient_dataset['trt_the'] == 1) |
                            (df_patient_dataset['trt_oth'] == 1)), 'group'] = 'Combined Treatment'

print(df_patient_dataset['group'].value_counts())


In [ ]:
from scipy.stats import ttest_ind

# Selecting the outcome variable for comparison
outcome_variable = 'cgis_change'  # or 'gaf_lv' based on your choice

# Grouping the data
group1 = df_patient_dataset[df_patient_dataset['group'] == 'Antidepressants Only'][outcome_variable]
group2 = df_patient_dataset[df_patient_dataset['group'] == 'Combined Treatment'][outcome_variable]

# Performing t-test
t_stat, p_value = ttest_ind(group1, group2, nan_policy='omit')

print(f'T-test results: Statistic={t_stat}, P-value={p_value}')


### T-test Results Interpretation

- **T-statistic**: 1.7451
- **P-value**: 0.0811

#### What Does This Mean?

1. **T-Statistic (1.7451)**:
   - This value, derived from your t-test, measures the difference between the two groups (Antidepressants Only vs. Combined Treatment) in terms of standard error. A t-statistic of 1.7451 indicates a moderate difference between the average CGIS score changes of these groups.
   - The t-statistic is positive, suggesting that the mean change in CGIS scores in the 'Combined Treatment' group might be higher than in the 'Antidepressants Only' group, although this is not yet confirmed as significant.

2. **P-Value (0.0811)**:
   - The p-value is a measure of the probability that the observed results (or more extreme) would occur if there were no actual difference between the groups (null hypothesis). 
   - With a p-value of 0.0811, your results do not reach the conventional threshold for statistical significance (usually set at 0.05). This means that, based on your dataset and the t-test performed, the observed difference in treatment outcomes is not statistically significant at the 5% level.
   - However, the p-value is relatively close to the significance threshold, which might suggest a possible trend or effect that could warrant further investigation, particularly with a larger sample size or different methodology.

In [ ]:
# Analyzing treatment effectiveness across age groups
age_group_effectiveness = df_patient_dataset.groupby(['age_group', 'group'])['cgis_change'].mean().unstack()
print(age_group_effectiveness)

### Interpretation of the Results:
- The mean change in CGIS scores across different age groups for both "Antidepressants Only" and "Combined Treatment" groups is provided.
- For age groups with data, you can see how the average change in CGIS scores varies. This change is a proxy for the improvement or worsening of the condition.

In [ ]:
# Example: Analyzing the impact of anxiety history on treatment choice
anxiety_treatment_analysis = df_patient_dataset.groupby(['medical_history_anx', 'group']).size().unstack()
print(anxiety_treatment_analysis)


- **Anxiety Disorder Influence**:
  - Without Anxiety Disorder (medical_history_anx = 0): 
    - Antidepressants Only: 347 patients
    - Combined Treatment: 1814 patients
  - With Anxiety Disorder (medical_history_anx = 1): 
    - Antidepressants Only: 142 patients
    - Combined Treatment: 697 patients

## Creating different types of treatment groups.

In [ ]:
df_patient_dataset['treatment_combination'] = df_patient_dataset.apply(create_treatment_combination, axis=1)

In [ ]:
mdd_summary = df_patient_dataset['treatment_combination'].value_counts()
mdd_summary